## 0. Libraries import

In [8]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [9]:
# imports 
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
#train test split
from sklearn.model_selection import train_test_split
# Hyperparameters selection
from sklearn.model_selection import RandomizedSearchCV
# Importing cross validation function from sklearn
from sklearn.model_selection import cross_val_score
# models
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
# error
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

## 1. Data import

### 1.1 Impor train data

In [10]:
df_diamonds_train=pd.read_csv('../data/diamonds_train.csv')
df_diamonds_train.pop("Unnamed: 0") #dropped an unnecessary column
df_diamonds_train

,index_id,depth,table,x,y,z,price,carat,cut,color,clarity,city
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,4268,1.21,Premium,J,VS2,Dubai
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75,505,0.32,Very Good,H,VS2,Kimberly
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,65.5,55.0,5.62,5.53,3.65,2686,0.71,Fair,G,VS1,Las Vegas
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,63.8,56.0,4.68,4.72,3.00,738,0.41,Good,D,SI1,Kimberly
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,60.5,59.0,6.55,6.51,3.95,4882,1.02,Ideal,G,SI1,Dubai
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,62.7,57.0,7.10,7.04,4.43,10070,1.34,Ideal,G,VS1,Antwerp
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,57.1,60.0,8.31,8.25,4.73,12615,2.02,Good,F,SI2,Madrid
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,62.7,56.0,6.37,6.42,4.01,5457,1.01,Ideal,H,SI1,Kimberly
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,61.9,54.3,4.45,4.47,2.76,456,0.33,Ideal,J,VS1,Kimberly


### 1.2 Impor test data

In [11]:
df_diamonds_test=pd.read_csv('../data/diamonds_test.csv')
df_diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


## 2. Data preparation (training set)

### 2.1 Defining numerical and categorical features

In [12]:
# Excluding city (with trees sometimes is better to keep variables even if they are correlated)
num_features_list=['x','y','z','depth','table','carat']
cat_features_list=['cut','color','clarity','city']
target='price'
features_list=['x','y','z','depth','table','carat','cut','color','clarity','city']

## 3. Feature engineering (training set)

### 3.1 Target encoding 

In [13]:
# 1.Target encoding for categorical variables
# Mean
cut_encoding = df_diamonds_train.groupby(['cut'])['price'].mean().to_dict()
df_diamonds_train['cut_encoding'] = df_diamonds_train['cut'].map(cut_encoding).astype(float)
color_encoding = df_diamonds_train.groupby(['color'])['price'].mean().to_dict()
df_diamonds_train['color_encoding'] = df_diamonds_train['color'].map(color_encoding).astype(float)
clarity_encoding = df_diamonds_train.groupby(['clarity'])['price'].mean().to_dict()
df_diamonds_train['clarity_encoding'] = df_diamonds_train['clarity'].map(clarity_encoding).astype(float)

In [14]:
features_list_encoding=['x','y','z','depth','table','carat','cut','color','clarity','city','cut_encoding','color_encoding'
                        ,'clarity_encoding']

### 3.2 Category encoding

In [15]:
# 3. Defining features y target
X=df_diamonds_train[features_list_encoding]
y=df_diamonds_train['price']

In [16]:
for column in cat_features_list:
    X[column]=X[column].astype('category')
    X[column]=X[column].cat.codes

In [17]:
X

,x,y,z,depth,table,carat,cut,color,clarity,city,cut_encoding,color_encoding,clarity_encoding
0,6.83,6.79,4.25,62.4,58.0,1.21,3,6,5,2,4617.322612,5346.234112,3913.590182
1,4.35,4.38,2.75,63.0,57.0,0.32,4,4,5,3,3994.444420,4476.469014,3913.590182
2,5.62,5.53,3.65,65.5,55.0,0.71,0,3,4,4,4333.271980,4023.214902,3796.813551
3,4.68,4.72,3.00,63.8,56.0,0.41,1,0,2,3,3880.611794,3134.943157,3999.856908
4,6.55,6.51,3.95,60.5,59.0,1.02,2,3,2,2,3436.112577,4023.214902,3999.856908
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,7.10,7.04,4.43,62.7,57.0,1.34,2,3,4,1,3436.112577,4023.214902,3796.813551
40451,8.31,8.25,4.73,57.1,60.0,2.02,1,2,3,7,3880.611794,3677.355720,5101.044307
40452,6.37,6.42,4.01,62.7,56.0,1.01,2,4,2,3,3436.112577,4476.469014,3999.856908
40453,4.45,4.47,2.76,61.9,54.3,0.33,2,6,4,3,3436.112577,5346.234112,3796.813551


### 3.3 Define train and validation

In [18]:
# 5.Splitting train and test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## 5. Feature engineering (test set) 

In [22]:
# 0. Adapting categorical features for validation model
X_test=df_diamonds_test[features_list]

### 5.1 Target encoding (mean and std)

In [23]:
# 1.Target encoding for categorical variables
# Mean
cut_encoding = df_diamonds_train.groupby(['cut'])['price'].mean().to_dict()
X_test['cut_encoding'] = X_test['cut'].map(cut_encoding).astype(float)
color_encoding = df_diamonds_train.groupby(['color'])['price'].mean().to_dict()
X_test['color_encoding'] = X_test['color'].map(color_encoding).astype(float)
clarity_encoding = df_diamonds_train.groupby(['clarity'])['price'].mean().to_dict()
X_test['clarity_encoding'] = X_test['clarity'].map(clarity_encoding).astype(float)

In [24]:
X_train.shape

(32364, 13)

In [25]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32364 entries, 32121 to 15795
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   x                 32364 non-null  float64
 1   y                 32364 non-null  float64
 2   z                 32364 non-null  float64
 3   depth             32364 non-null  float64
 4   table             32364 non-null  float64
 5   carat             32364 non-null  float64
 6   cut               32364 non-null  int8   
 7   color             32364 non-null  int8   
 8   clarity           32364 non-null  int8   
 9   city              32364 non-null  int8   
 10  cut_encoding      32364 non-null  float64
 11  color_encoding    32364 non-null  float64
 12  clarity_encoding  32364 non-null  float64
dtypes: float64(9), int8(4)
memory usage: 2.6 MB


### 5.2 Change categorical variables to code

In [26]:
for column in cat_features_list:
    X_test[column]=X_test[column].astype('category')
    X_test[column]=X_test[column].cat.codes

In [27]:
X_test

,x,y,z,depth,table,carat,cut,color,clarity,city,cut_encoding,color_encoding,clarity_encoding
0,5.82,5.89,3.67,62.7,60.0,0.79,4,2,2,0,3994.444420,3677.355720,3999.856908
1,6.81,6.89,4.18,61.0,57.0,1.20,2,6,4,10,3436.112577,5346.234112,3796.813551
2,7.38,7.32,4.57,62.2,61.0,1.57,3,4,2,3,4617.322612,4476.469014,3999.856908
3,6.09,6.13,3.90,63.8,54.0,0.90,4,2,2,3,3994.444420,3677.355720,3999.856908
4,5.05,5.09,3.19,62.9,58.0,0.50,4,2,4,0,3994.444420,3677.355720,3796.813551
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,5.35,5.32,3.30,61.9,56.0,0.57,2,1,2,0,3436.112577,3088.342526,3999.856908
13481,5.71,5.73,3.56,62.2,55.0,0.71,2,5,5,8,3436.112577,5090.868800,3913.590182
13482,5.75,5.71,3.53,61.6,55.0,0.70,2,2,4,11,3436.112577,3677.355720,3796.813551
13483,5.85,5.89,3.45,58.8,57.0,0.70,4,2,3,10,3994.444420,3677.355720,5101.044307


In [28]:
X_test.shape

(13485, 13)

In [29]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   x                 13485 non-null  float64
 1   y                 13485 non-null  float64
 2   z                 13485 non-null  float64
 3   depth             13485 non-null  float64
 4   table             13485 non-null  float64
 5   carat             13485 non-null  float64
 6   cut               13485 non-null  int8   
 7   color             13485 non-null  int8   
 8   clarity           13485 non-null  int8   
 9   city              13485 non-null  int8   
 10  cut_encoding      13485 non-null  float64
 11  color_encoding    13485 non-null  float64
 12  clarity_encoding  13485 non-null  float64
dtypes: float64(9), int8(4)
memory usage: 1001.0 KB


## Model definition - RandomForestRegressor - with Random Hyperparameter Grid

RandomForestRegressor: multiple trees in paralel changing samples and convining diferrent features (overfitting when the tree is big and good to reduce error variance)

Main Parameters:
   - bootstrap -> method for sampling data points (TRUE bagging and FALSE pasting, with/without replacement)
   - n_estimators -> number of trees in the foreset
   - max_depth -> max number of levels in each decision tree
   - max_features -> max number of features considered for splitting a node
   - ccp_alpha ->
   - criterion ->
   - max_leaf_nodes -> max number of solution nodes 
   - max_samples ->
   - min_impurity_decrease ->
   - min_samples_leaf -> min number of data points allowed in a leaf node
   - min_samples_split -> min number of data points placed in a node before the node is split
   - min_weight_fraction_leaf
   - n_estimators -> number of trees in the foreset
   - n_jobs 
   - oob_score 
   - random_state
   - verbose
   - warm_start   

In [30]:
# 1. XGBRegressor 
model = XGBRegressor(n_estimators=200,colsample_bylevel=1,colsample_bynode=1,
                     colsample_bytree=0.8,reg_alpha=1, reg_lambda=1,gamma=0,learning_rate=0.1, random_state=42)
hyperparameters = model.get_params()
print(type(model), '\n')

<class 'xgboost.sklearn.XGBRegressor'> 



## Model training with validation

In [31]:
%%time
# 1. XGBRegressor early_stopping_rounds=40
# Model training
model.fit(X_train, y_train,eval_set=[(X_train,y_train),(X_val,y_val)],early_stopping_rounds=40)

[0]	validation_0-rmse:5050.20361	validation_1-rmse:5089.46191
[1]	validation_0-rmse:4561.85986	validation_1-rmse:4599.20605
[2]	validation_0-rmse:4125.37353	validation_1-rmse:4160.49902
[3]	validation_0-rmse:3731.65332	validation_1-rmse:3765.33496
[4]	validation_0-rmse:3378.73560	validation_1-rmse:3411.42163
[5]	validation_0-rmse:3059.69751	validation_1-rmse:3091.21118
[6]	validation_0-rmse:2781.38159	validation_1-rmse:2810.80444
[7]	validation_0-rmse:2523.39648	validation_1-rmse:2552.75732
[8]	validation_0-rmse:2295.53052	validation_1-rmse:2323.95093
[9]	validation_0-rmse:2090.69482	validation_1-rmse:2118.79736
[10]	validation_0-rmse:1904.92310	validation_1-rmse:1932.43872
[11]	validation_0-rmse:1738.95666	validation_1-rmse:1766.02600
[12]	validation_0-rmse:1590.44519	validation_1-rmse:1617.73425
[13]	validation_0-rmse:1458.02942	validation_1-rmse:1486.18897
[14]	validation_0-rmse:1340.17981	validation_1-rmse:1368.88660
[15]	validation_0-rmse:1235.43701	validation_1-rmse:1263.91492
[1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [32]:
%%time
# Model predictions
y_pred_val = model.predict(X_val)

CPU times: user 49.5 ms, sys: 4.27 ms, total: 53.8 ms
Wall time: 11.5 ms


In [33]:
%%time
# Model predictions
y_pred_train = model.predict(X_train)

CPU times: user 129 ms, sys: 3.53 ms, total: 133 ms
Wall time: 24.8 ms


## Training set error

In [34]:
# Model predictions
rmse_train = mean_squared_error(y_train, y_pred_train)**0.5
rmse_train

447.4857359017463

In [35]:
mae_train=mean_absolute_error(y_train, y_pred_train)
mae_train

247.18925707379432

In [36]:
r2r = r2_score(y_val, y_pred_val)
r2r

0.9828280586119165

## Model validation

In [37]:
#432
rmse_val = mean_squared_error(y_val, y_pred_val)**0.5
rmse_val

528.8114576461136

In [38]:
mae_val=mean_absolute_error(y_val, y_pred_val)
mae_val

276.82993836120596

In [39]:
r2r = r2_score(y_val, y_pred_val)
r2r

0.9828280586119165

## Model training without validation

In [40]:
%%time
# Model training
model.fit(X, y,eval_set=[(X_train,y_train),(X_val,y_val)],early_stopping_rounds=40)

[0]	validation_0-rmse:5050.02051	validation_1-rmse:5086.54346
[1]	validation_0-rmse:4560.79883	validation_1-rmse:4596.78564
[2]	validation_0-rmse:4124.03711	validation_1-rmse:4157.51758
[3]	validation_0-rmse:3729.29858	validation_1-rmse:3761.69580
[4]	validation_0-rmse:3374.62305	validation_1-rmse:3406.22314
[5]	validation_0-rmse:3055.26245	validation_1-rmse:3086.02002
[6]	validation_0-rmse:2776.91675	validation_1-rmse:2804.64111
[7]	validation_0-rmse:2518.89331	validation_1-rmse:2545.64551
[8]	validation_0-rmse:2289.88916	validation_1-rmse:2313.61377
[9]	validation_0-rmse:2086.11768	validation_1-rmse:2108.48804
[10]	validation_0-rmse:1900.31494	validation_1-rmse:1921.19141
[11]	validation_0-rmse:1734.93152	validation_1-rmse:1753.57800
[12]	validation_0-rmse:1587.10584	validation_1-rmse:1605.68176
[13]	validation_0-rmse:1454.63806	validation_1-rmse:1473.25513
[14]	validation_0-rmse:1337.58752	validation_1-rmse:1355.24036
[15]	validation_0-rmse:1233.05481	validation_1-rmse:1250.19080
[1

[134]	validation_0-rmse:422.11838	validation_1-rmse:422.50534
[135]	validation_0-rmse:422.06284	validation_1-rmse:422.40808
[136]	validation_0-rmse:421.78479	validation_1-rmse:422.28607
[137]	validation_0-rmse:420.96139	validation_1-rmse:422.02811
[138]	validation_0-rmse:420.36871	validation_1-rmse:421.57956
[139]	validation_0-rmse:420.28583	validation_1-rmse:421.47900
[140]	validation_0-rmse:419.96643	validation_1-rmse:421.20959
[141]	validation_0-rmse:419.70294	validation_1-rmse:420.93365
[142]	validation_0-rmse:419.43716	validation_1-rmse:420.83466
[143]	validation_0-rmse:418.68454	validation_1-rmse:420.27155
[144]	validation_0-rmse:418.22919	validation_1-rmse:420.05045
[145]	validation_0-rmse:417.44641	validation_1-rmse:419.29245
[146]	validation_0-rmse:416.70544	validation_1-rmse:418.66391
[147]	validation_0-rmse:416.65277	validation_1-rmse:418.61050
[148]	validation_0-rmse:415.80582	validation_1-rmse:417.93521
[149]	validation_0-rmse:415.26651	validation_1-rmse:417.40268
[150]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

## Test Preditions

In [41]:
predictions = model.predict(X_test)

In [42]:
predictions=pd.DataFrame(predictions)

In [43]:
predictions.reset_index(inplace=True)

In [44]:
predictions=predictions.rename({0: 'price','index': 'id'}, axis=1)

## Save Preditions

In [45]:
predictions.to_csv('../data/diamonds_predictions_cat_target_codes_XGB.csv',index=False)